In [6]:
# Import all necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2 as cv
import re
import requests

from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [7]:
test_df = pd.read_csv('.clothes_dataset/test.csv')
train_df = pd.read_csv('.clothes_dataset/train.csv')
val_df = pd.read_csv('.clothes_dataset/val.csv')

In [11]:
X = []
Y = []

img = test_df['image']
test_df

,Unnamed: 0,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes
0,0,./clothes_dataset\blue_shorts\1ec871cdfe41ff20...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,./clothes_dataset\black_pants\0f8eb4509cea01f9...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,./clothes_dataset\white_shoes\1d70ebaec12e09f3...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,3,./clothes_dataset\green_shoes\a4e155f7249a591e...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,./clothes_dataset\blue_shirt\2edd17185a4f1b8d9...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,3411,./clothes_dataset\black_shirt\d76faa497083f1f6...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3412,3412,./clothes_dataset\blue_pants\691dacb1274608390...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3413,3413,./clothes_dataset\blue_pants\9b5ac8ce3de1d5219...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3414,3414,./clothes_dataset\black_pants\7db76af76fe19a01...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
# Lists to store images data and labels
X = []
Y = []

# Input shape for ANN and also to resize images
input_shape = (96, 96, 3)

# Merge train and test data into one dataset
# Loop through train and test subsets
for subset in ('train', 'test'):
    path_to_subset = f'.apparel-images-dataset/{subset}'

    # Loop through all classes in subset
    for folder in os.listdir(path_to_subset):

        # Loop through all images inh each class
        for image in os.listdir(os.path.join(path_to_subset, folder)):
            # Defining path to image
            path_to_image = os.path.join(path_to_subset, folder, image)
            # Reading image using cv2
            image = cv.imread(path_to_image)
            # Resizing image
            image = cv.resize(image, (input_shape[1], input_shape[0]))
            # Extracting labels from 'path_to_image' using regex
            label = re.findall(r'\w+\_\w+', path_to_image)[0].split('_')

            # Appending data and labels to X and Y lists
            X.append(image)
            Y.append(label)

# Convert X to numpy array and scale values between 0 and 1
X = np.array(X) / 255.0
Y = np.array(Y)

# Binarize labels
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)